In [1]:
!pip3 install -U scipy

!git clone https://github.com/jnordberg/tortoise-tts.git
%cd tortoise-tts
!pip3 install transformers==4.19.0 fastapi[all] nest-asyncio pyngrok uvicorn python-multipart==0.0.4
!pip3 install -r requirements.txt
!python3 setup.py install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Cloning into 'tortoise-tts'...
remote: Enumerating objects: 1481, done.
remote: Total 1481 (delta 0), reused 0 (delta 0), pack-reused 1481
Receiving objects: 100% (1481/1481), 53.58 MiB | 20.29 MiB/s, done.
Resolving deltas: 100% (590/590), done.
/content/tortoise-tts
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 15.5 MB/s 
     |████████████████████████████████| 55 kB 4.7 MB/s 
     |████████████████████████████████| 745 kB 56.2 MB/s 
     |████████████████████████████████| 56 kB 5.2 MB/s 
     |████████████████████████████████| 6.6 MB 46.1 MB/s 
     |████████████████████████████████| 163 kB 47.0 MB/s 
     |████████████████████████████████| 58 kB 4.8 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |███████████████

In [ ]:
!ngrok authtoken YOUR_TOKEN          #Please get your token by singing up at ngrok.com and replace by YOUR_TOKEN

In [2]:
# Imports used through the rest of the notebook.
import torch
import torchaudio
import torch.nn as nn
import torch.nn.functional as F

import IPython

from tortoise.api import TextToSpeech
from tortoise.utils.audio import load_audio

# This will download all the models used by Tortoise from the HuggingFace hub.
tts = TextToSpeech()

Downloading:   0%|          | 0.00/2.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.18G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/181 [00:00<?, ?B/s]

...

Done.


Done.


Done.


Done.


Done.


In [3]:
def load_voice(paths):
  conds = []
  for cond_path in paths:
      c = load_audio(cond_path, 22050)
      conds.append(c)
  return conds, None

In [4]:
def clone_voice(name, voice_paths, text):
  # Generate speech with the custotm voice.
  voice_samples, conditioning_latents = load_voice(voice_paths)
  gen = tts.tts_with_preset(text, voice_samples=voice_samples, conditioning_latents=conditioning_latents, 
                            preset="fast")
  torchaudio.save(f'generated-{name}.wav', gen.squeeze(0).cpu(), 24000)
  return f'generated-{name}.wav'

In [6]:
from fastapi import FastAPI,File, UploadFile, Form
from fastapi.responses import FileResponse
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import uuid
import os

app = FastAPI()

@app.get("/")
async def index():
    return {"Status": "Running"}

@app.post("/get_audio")
async def get_audio(
    your_name: str = Form(default=""),
    text: str = Form(default=""),
    audio1: UploadFile = File(...),
    audio2: UploadFile = File(...)
    ):
    # if not os.path.isdir("./tortoise/voices/" + your_name):
    #     os.mkdir("./tortoise/voices/" + your_name)
    print("Start")
    try:
        file_name1 = str(uuid.uuid4()) + ".wav"
        with open(file_name1, "wb") as f:
            content = audio1.file.read()
            f.write(content)
    except Exception as e:
        return {"response": False, "msg": str(e)}
    print("Done")
    try:
        file_name2 = str(uuid.uuid4()) + ".wav"
        with open(file_name2, "wb") as f:
            content = audio2.file.read()
            f.write(content)
    except Exception as e:
        return {"response": False, "msg": str(e)}

    file_name = clone_voice(your_name, [file_name1, file_name2], text)
    return FileResponse(path=file_name, media_type='application/octet-stream', filename=file_name)
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
print("Please Copy above public url and past into config.py file of UI app.")
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [79]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: http://ee6907c0ea28.ngrok.io


Start
Done
Generating autoregressive samples..


100%|██████████| 6/6 [00:13<00:00,  2.30s/it]


Computing best candidates using CLVP and CVVP


...

100%|██████████| 6/6 [00:09<00:00,  1.53s/it]


Transforming autoregressive outputs into audio..


100%|██████████| 80/80 [00:09<00:00,  8.22it/s]


INFO:     34.70.170.35:0 - "POST /get_audio HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [79]
